In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import psycopg2
con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
port= '5439', user= 'mantix', password= 'Mantix123!')
cur = con.cursor()

In [ ]:
query = """(SELECT DISTINCT customer_account_id,
                               DATEDIFF(DAY,MAX(submitted_date),getdate()) AS days_since_last_transaction,
                               COUNT(*) AS toal_purchases,
                               SUM(total) AS total_amount
                        FROM cdp.orders_data
                        WHERE tenant_id = 'TNB00001'
                        AND   store_id = 'STOM000000001'
                        AND    status NOT IN ('Abandoned','Errored','Pending','PendingReview','Cancelled','null')
                        GROUP BY customer_account_id)"""

In [ ]:
b = pd.read_sql(query,con)

In [ ]:
b

In [ ]:
b['total_amount'] = b['total_amount'].fillna(0)

In [ ]:
row_indexes=b[b['days_since_last_transaction']>=30].index
b.loc[row_indexes,'churn']= 1


In [ ]:
b['churn'] = b['churn'].fillna(0)

In [ ]:
b.churn.value_counts()

In [ ]:
b['churn'] = b['churn'].astype('category')

In [ ]:
response = pd.DataFrame()
response['count'] = b.churn.value_counts()
response['percentage'] = (response['count'] / response['count'].sum()) * 100
print(response)

In [ ]:
b.shape

In [ ]:
b = b.dropna()

In [ ]:
b.shape

In [ ]:
b.head()

In [ ]:
#setting our training and test set
training_set = b.drop(['churn','days_since_last_transaction','customer_account_id'], axis=1)
target_var = b['churn'].values

In [ ]:
training_set.shape

In [ ]:
#splitting our data...
from sklearn.model_selection import train_test_split
xtrain,xtest, ytrain, ytest = train_test_split(training_set, target_var, test_size=0.3, random_state=0)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report
gbr = GradientBoostingClassifier(n_estimators = 500)
gbr.fit(xtrain,ytrain)
prediction_gbr = gbr.predict(xtest)
print("Training accuracy: %.2f" %(gbr.score(xtrain, ytrain) * 100))
print("Test accuracy    : %.2f" %(gbr.score(xtest, ytest) * 100))
print("")
print("Confusion matrix..")
print(confusion_matrix(ytest, prediction_gbr))
print("")
print(classification_report(ytest, prediction_gbr))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 500)
random_forest.fit(xtrain,ytrain)
prediction_rf = random_forest.predict_proba(xtest)
print("Training accuracy: %.2f" %(random_forest.score(xtrain, ytrain) * 100))
print("Test accuracy    : %.2f" %(random_forest.score(xtest, ytest) * 100))
print("")
print("Confusion matrix..")
print("")

In [ ]:
import joblib
joblib.dump(random_forest, 'Bimart_churn.pkl') 
rf_prop = joblib.load('Bimart_churn.pkl') 

In [ ]:
prediction_rf = random_forest.predict_proba(training_set)

In [ ]:
prediction_rf_df = pd.DataFrame(data = prediction_rf)

In [ ]:
prediction_rf_df.head()

In [ ]:
prediction_rf_df = prediction_rf_df.rename(columns={prediction_rf_df.columns[1]: 'churn_prediction'})

In [ ]:
prediction_rf_df.tail()

In [ ]:
prediction_rf_df.shape

In [ ]:
b.shape

In [ ]:
# b['churn_prediction'] = prediction_rf_df.iloc[1]

In [ ]:
b['churn_prediction'] = prediction_rf_df['churn_prediction']

In [ ]:
b.head()

In [ ]:
# prediction_rf_df['customer_account_id'] = prediction_rf_df.iloc[0]

In [ ]:
# prediction_rf_df.head()

In [ ]:
# new_df = b.merge(prediction_rf_df, left_index=True, right_index=True, how='inner')

In [ ]:
# results_df = new_df[['customer_account_id_x','churn_prediction']]

In [ ]:
new_df=b[['customer_account_id','churn_prediction']]

In [ ]:
# results_df = results_df.rename(columns={'customer_account_id_x': 'customer_account_id'})

In [ ]:
new_df.tail()

In [ ]:
new_df['churn_prediction'] = new_df['churn_prediction'].mul(100).round(2)

In [ ]:
new_df.tail()

In [ ]:
new_df['tenant_id'] = 'TNB00001'
new_df['store_id'] = 'STOM000000001'

In [ ]:
import datetime
from datetime import date
date = date.today()
date

In [ ]:
new_df['create_date']= date

In [ ]:
new_df.head()
results_df=new_df.copy()

In [ ]:
results_df.tail()

In [ ]:
def load_db(results_df):
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()

    def drop_table(conn,table):
        try:
            query = f"""DROP TABLE IF EXISTS {table}"""
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            print(f'Drop table {table} successfully')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            #return 1

    def create_table(conn,table):
        try:
            query_col = """(customer_account_id VARCHAR(1000),
                churn_prediction FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_date VARCHAR(1000))"""
            query = f"""
                    CREATE TABLE {table} 
                    """+query_col
            cur = conn.cursor()
            cur.execute("ROLLBACK")
            conn.commit()
            cur.execute(query)
            conn.commit()
            print(f'Successfully created the table {table}')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            return 1
            
    def single_insert(conn, insert_req):
        """ Execute a single INSERT request """
        cursor = conn.cursor()
        try:
            cursor.execute(insert_req)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
        
    def batch_insert(conn, batch_df):
        # Create a cursor object from the connection
        cursor = conn.cursor()
        statment= f"""INSERT into {table}(customer_account_id,churn_prediction,tenant_id,store_id,create_date) values(%s,%s,%s,%s,%s);"""
        try:
            # Execute the batch INSERT request
            cursor.executemany(statment, batch_df)
            # Commit the changes to the database
            conn.commit()
            print("Batch INSERT successful.")
        except (Exception, psycopg2.DatabaseError) as error:
            # Rollback the transaction in case of an error
            conn.rollback()
            print("Batch INSERT failed, Error:", error)
        finally:
            # Close the cursor
            cursor.close() 
        
        
    def insert_dataframe(final_df,conn,table):
        batch_df=[]
        for i in final_df.index:
            current_tuple=(
                        str(results_df.loc[i]['customer_account_id']), 
                        float(results_df.loc[i]['churn_prediction']),
                        str(results_df.loc[i]['tenant_id']),
                        str(results_df.loc[i]['store_id']),
                        str(results_df.loc[i]['create_date'])
                )
            batch_df.append(current_tuple)
            if (len(batch_df) == 500): 
                batch_insert(conn, batch_df)
                batch_df=[]
                print("Current index: {}, time: {}".format(i, datetime.datetime.now()))
            #endif
        # Insert any remaining rows
        if (len(batch_df) > 0):
            batch_insert(conn,batch_df)
        print('Successfully inserted dataframe into the table')
        
    table = 'cdp.churn'
    drop_table(con,table)
    create_table(con,table)
    startTime = datetime.datetime.now()
    insert_dataframe(results_df,con,table)
    print("Start: {}, Finish: {}".format(startTime, datetime.datetime.now()))

if __name__ == '__main__':


    # load data into database
    load_db(results_df)

######################################

In [ ]:
# def load_db(results_df):
#     con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
#     port= '5439', user= 'mantix', password= 'Mantix123!')
#     cur = con.cursor()

#     def drop_table(conn,table):
#         try:
#             query = f"""DROP TABLE IF EXISTS {table}"""
#             cur = conn.cursor()
#             cur.execute(query)
#             conn.commit()
#             print(f'Drop table {table} successfully')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             #return 1

#     def create_table(conn,table):
#         try:
#             query_col = """(customer_account_id VARCHAR(1000),
#                 churn_prediction FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_date VARCHAR(1000))"""
#             query = f"""
#                     CREATE TABLE {table} 
#                     """+query_col
#             cur = conn.cursor()
#             cur.execute("ROLLBACK")
#             conn.commit()
#             cur.execute(query)
#             conn.commit()
#             print(f'Successfully created the table {table}')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             return 1
            
#     def single_insert(conn, insert_req):
#         """ Execute a single INSERT request """
#         cursor = conn.cursor()
#         try:
#             cursor.execute(insert_req)
#             conn.commit()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cursor.close()
#             return 1
#         cursor.close()
        
#     def insert_dataframe(results_df,conn,table):
#         for i in results_df.index:
#             query = f"""
#                     INSERT into {table}(customer_account_id,churn_prediction,tenant_id,store_id,create_date) 
#                         values('%s',%s,%s,%s,%s);
#                     """ % (results_df.loc[i]['customer_account_id'], 
#                         results_df.loc[i]['churn_prediction'],
#                          "'"+str(results_df.loc[i]['tenant_id'])+"'",
#                         "'"+str(results_df.loc[i]['store_id'])+"'","'"+str(results_df.loc[i]['create_date'])+"'")
#             if i%50==0:
#                 print(i,end=' ')
#             single_insert(conn, query)
#         conn.commit()
#         print('\nSuccessfully inserted dataframe into the table')
        
#     table = 'cdp.churn'
#     drop_table(con,table)
#     create_table(con,table)
#     insert_dataframe(results_df,con,table)

# if __name__ == '__main__':


#     # load data into database
#     load_db(results_df)

# ######################################

In [ ]:
# def load_db(cltv_final):
#     con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
#     port= '5439', user= 'mantix', password= 'Mantix123!')
#     cur = con.cursor()

#     def drop_table(conn,table):
#         try:
#             query = f"""DROP TABLE IF EXISTS {table}"""
#             cur = conn.cursor()
#             cur.execute(query)
#             conn.commit()
#             print(f'Drop table {table} successfully')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             #return 1

#     def create_table(conn,table):
#         try:
#             query_col = """(customer_account_id VARCHAR(1000),
#                 cltv_6M FLOAT,cltv_12M FLOAT,
#                 cltv_24M FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
#             query = f"""
#                     CREATE TABLE {table} 
#                     """+query_col
#             cur = conn.cursor()
#             cur.execute("ROLLBACK")
#             conn.commit()
#             cur.execute(query)
#             conn.commit()
#             print(f'Successfully created the table {table}')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             return 1
            
#     def single_insert(conn, insert_req):
#         """ Execute a single INSERT request """
#         cursor = conn.cursor()
#         try:
#             cursor.execute(insert_req)
#             conn.commit()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cursor.close()
#             return 1
#         cursor.close()
        
#     def insert_dataframe(cltv_final,conn,table):
#         for i in cltv_final.index:
#             query = f"""
#                     INSERT into {table}(customer_account_id,cltv_6M,cltv_12M,cltv_24M,tenant_id,store_id,create_dms) 
#                         values('%s',%s,%s,%s,%s,%s,%s);
#                     """ % (cltv_final.loc[i]['CustomerId'], 
#                         cltv_final.loc[i]['cltv_6M'],
                        
#                         cltv_final.loc[i]['cltv_12M'],
                        
#                         cltv_final.loc[i]['cltv_24M'],
                        
#                         "'"+str(cltv_final.loc[i]['tenant_id'])+"'",
#                         "'"+str(cltv_final.loc[i]['store_id'])+"'",
#                          "'"+str(cltv_final.loc[i]['create_dms'])+"'")
#             if i%50==0:
#                 print(i,end=' ')
#             single_insert(conn, query)
#         conn.commit()
#         print('\nSuccessfully inserted dataframe into the table')
        
#     table = 'cdp.cltv'
#     drop_table(con,table)
#     create_table(con,table)
#     insert_dataframe(cltv_final,con,table)

# if __name__ == '__main__':

#     # start cltv
#     cltv_final = start_cltv()

#     # load data into database
#     load_db(cltv_final)
